# Density and public transportation

Sufficient population and job density are important for a public transportation system. Therefore plotting densities and public transportation systems on the same map can reveal opportuntities to build more public transportation lines (where density is high, but there are no existing lines) and to build more housing/jobs (where public transportation lines exist, but densities are low).

Let's create a map of population density. In the United States, we can get this geo-tagged population data from the United States Census Bureau. dpd.modeling has a class Zones to store this data and a method to automatically pull this data from the United States Census Bureau.

First, we'll define the state that we are interested in mapping. States seems to be the best level to get data for. If we get data by county, there are lots of requests so it takes too long. However, data for the whole country is unnecessary.

In [ ]:
import us
from ipywidgets import Select
from IPython.display import display

YEAR = "2017"

state = Select(
    options=list(map(lambda x: x.name, us.STATES)),
    description="State",
    value="California",
)
display(state)

Now we can get the census data for California. The government assigns a number to each state so California is 06. B01003_001E is the population in each census tract. This is renamed to Total Population. Also, ALAND is the land area of the zone. We use these two values to compute a density as population per area. Also, we simplify the geometry column to speed up any geometric computations.

In [ ]:
from dpd.modeling import Zones

zones = Zones.from_uscensus(str(us.states.lookup(state.value).fips), YEAR)

zones["geometry"] = zones["geometry"].apply(lambda x: x.simplify(0.001))

zones.head(1)

Here we can plot the population density of California.

In [ ]:
zones.explore(
    column="Total Population + Worker Population Density", scheme="JenksCaspall", k=20
)

For the rest of the notebook, we will focus on Los Angeles County. We can query OpenStreetMap for the geometric data that represents Los Angeles County and then combine it with our Zones to reduce the amount of data to process

In [ ]:
from shapely.geometry import Polygon
from shapely.ops import linemerge

from dpd.osm import OSM

relation = 396479  # Los Angeles County
osm = OSM()
osm.download_relation(relation)
ways = [
    osm.ways[member["ref"]].geo
    for member in osm.relations[relation]["members"]
    if member["type"] == "way"
]
ways

ways_merged = linemerge(ways)
longest_length = 0
for way in ways_merged.geoms:
    if way.length > longest_length:
        longest_length = way.length
        longest_way = way
los_angeles_county = Polygon(longest_way)

los_angeles_county

In [ ]:
zones.geometry.within(los_angeles_county).value_counts()

In [ ]:
zones = zones[zones.geometry.within(los_angeles_county)]

Now we can use h3fy to recompute our population density into hexigons. This normalizes the geometries so they can be used for calculations later on.

In [ ]:
from tobler.util import h3fy
from tobler.area_weighted import area_interpolate

zones.to_crs(epsg=4087, inplace=True)
h3_zones = h3fy(zones, buffer=True)

interpolated = area_interpolate(
    source_df=zones,
    target_df=h3_zones,
    intensive_variables=["Total Population", "Worker Population", "ALAND"],
)
zones_interpolated = Zones(interpolated)
zones_interpolated["Total Population Density"] = (
    zones_interpolated["Total Population"] / zones_interpolated["ALAND"]
)
zones_interpolated["Worker Population Density"] = (
    zones_interpolated["Worker Population"] / zones_interpolated["ALAND"]
)
zones_interpolated["Total Population + Worker Population"] = (
    zones_interpolated["Total Population"] + zones_interpolated["Worker Population"]
)
zones_interpolated["Total Population + Worker Population Density"] = (
    zones_interpolated["Total Population + Worker Population"]
    / zones_interpolated["ALAND"]
)
zones_interpolated.head(1)

And we can plot the hexigons.

In [ ]:
zones_interpolated.explore(
    column="Total Population + Worker Population Density", scheme="JenksCaspall", k=20
)

These hexigons are pretty big. Let's recompute them to be smaller and plot with lonboard. Lonboard is great for plotting lots of data quickly.

In [ ]:
import numpy
from lonboard import Map, SolidPolygonLayer
from mapclassify import JenksCaspall
from palettable.matplotlib import Viridis_20

h3_zones = h3fy(zones, resolution=8, buffer=True)

interpolated = area_interpolate(
    source_df=zones,
    target_df=h3_zones,
    intensive_variables=["Total Population", "Worker Population", "ALAND"],
)
zones_interpolated = Zones(interpolated)
zones_interpolated["Total Population Density"] = (
    zones_interpolated["Total Population"] / zones_interpolated["ALAND"]
)
zones_interpolated["Worker Population Density"] = (
    zones_interpolated["Worker Population"] / zones_interpolated["ALAND"]
)
zones_interpolated["Total Population + Worker Population"] = (
    zones_interpolated["Total Population"] + zones_interpolated["Worker Population"]
)
zones_interpolated["Total Population + Worker Population Density"] = (
    zones_interpolated["Total Population + Worker Population"]
    / zones_interpolated["ALAND"]
)

layer = SolidPolygonLayer.from_geopandas(zones_interpolated, opacity=0.1)
jc = JenksCaspall(
    zones_interpolated["Total Population + Worker Population Density"].fillna(0), k=10
)
colors = []
for color in jc.yb:
    colors.append(Viridis_20.colors[color])
layer.get_fill_color = numpy.uint8(colors)

Map(layers=[layer])

Now, it would be helpful to do the same exercise with job densiites. However, the US Census Bureau does not provide this information ("Worker Population": "B08604_001E") at the tract level like they do for population. This leaves us with two options which both require large downloads.

1. We can download zip code worker populations.
2. Or we can download LODES data which includes origin-destination information.

We'll take a look at option 2 below. The LODES data is divided into three files: residential data, work data, and origin-destination data. There is also a cross-walk file that includes a translation from LODES GEOIDs to census tracts.

In [ ]:
from dpd.modeling import TripDataFrame

"""
od = TripDataFrame.from_lodes(us.states.lookup(state.value).abbr.lower(), YEAR)

od.head()"""

We can then combine the LODES data with our original output DataFrame (which includes the geometry) to add a job_density column.

In [ ]:
"""zones = Zones(zones).production_attraction_from_lodes(od)
h3_zones = h3fy(zones, resolution=9, buffer=True)

interpolated = area_interpolate(
    source_df=zones,
    target_df=h3_zones,
    intensive_variables=["Total Population", "Worker Population", "ALAND"],
)
zones_interpolated = Zones(interpolated)
zones_interpolated["Total Population Density"] = (
    zones_interpolated["Total Population"] / zones_interpolated["ALAND"]
)
zones_interpolated["Worker Population Density"] = (
    zones_interpolated["Worker Population"] / zones_interpolated["ALAND"]
)
zones_interpolated["Total Population + Worker Population"] = (
    zones_interpolated["Total Population"] + zones_interpolated["Worker Population"]
)
zones_interpolated["Total Population + Worker Population Density"] = (
    zones_interpolated["Total Population + Worker Population"]
    / zones_interpolated["ALAND"]
)

zones_interpolated.head(1)"""

And we can plot the job density like we ploted the population density above.

Los Angeles County Metro is planning to extend the C Line south towards Torrance. One of the design decisions is where to put a station: at the South Bay Galleria or Redondo Beach Transit Center? Let's look at the population density around these two locations. We can buffer the stop locations with a radius of 1000km and plot along with the density.

In [ ]:
from folium import Map
from geopandas import GeoSeries
from shapely.geometry import Point

station_locations = GeoSeries(
    data=[
        Point(-118.35884733230867, 33.86557221753289),
        Point(-118.35253437469025, 33.87283256915632),
    ],
    index=["South Bay Galleria", "Redondo Beach Transit Center"],
    crs="EPSG:4326",
).to_crs(epsg=4087)

folium_map = Map(location=(33.87, -118.35), zoom_start=14)
station_locations.buffer(1000).explore(m=folium_map)
zones.explore(
    column="Total Population + Worker Population Density",
    scheme="JenksCaspall",
    k=20,
    m=folium_map,
)

In [ ]:
from dpd.modeling import contour_dataframe

folium_map = Map(location=(33.87, -118.35), zoom_start=14)
zones.explore(
    column="Total Population + Worker Population Density",
    scheme="JenksCaspall",
    k=20,
    m=folium_map,
)
for station_location in station_locations:
    contour_dataframe(station_location, crs=station_locations.crs).explore(m=folium_map)
folium_map

In [ ]:
from dpd.modeling import DistanceDataFrame

distance_dataframe = DistanceDataFrame.from_origins_destinations(
    zones_interpolated.geometry.centroid,
    station_locations.geometry,
    method="distance",
)
distance_dataframe.columns = station_locations.index
distance_dataframe

In [ ]:
from matplotlib import pyplot as plt

fig = plt.figure(figsize=(18, 16))
ax = fig.add_subplot(111)
(distance_dataframe / 1.35).hist(
    weights=zones_interpolated["Total Population + Worker Population"],
    range=(0, 900),
    bins=30,
    cumulative=True,
    sharey=True,
    ax=ax,
)
ax.set_ylabel("Population (cumulative)")
ax.set_xlabel("Time (seconds)")

Now we download our public transportation systems so we can plot the lines. Here we can download the Los Angeles Metro Rail network from OpenStreetMap.

In [ ]:
from dpd.driving.network import Network

query = """
[out:json][timeout:25];
(
  relation["network"="Metro Rail"];

);
out body;
>;
out skel qt;
"""

network = Network.from_osm_query(query)

And we can plot the network on the original population density map.

In [ ]:
from lonboard import Map, ScatterplotLayer

layer = SolidPolygonLayer.from_geopandas(zones_interpolated, opacity=0.1)
jc = JenksCaspall(
    zones_interpolated["Total Population + Worker Population Density"], k=10
)
colors = []
for color in jc.yb:
    colors.append(Viridis_20.colors[color])
layer.get_fill_color = numpy.uint8(colors)
layers = [layer]
for route in network.routes:
    layers.append(
        ScatterplotLayer.from_geopandas(
            network.routes[route].stops, radius_min_pixels=10
        )
    )

Map(layers=layers)

In [ ]:
import folium

folium_map = folium.Map(location=(34, -118.3), zoom_start=11)
zones.explore(
    m=folium_map,
    column="Total Population + Worker Population Density",
    scheme="JenksCaspall",
    k=20,
)
for route in network.routes:
    network.routes[route].explore(m=folium_map)

folium_map

Now let's look at accessibility. We will download one metro line and compute the distance to each stop from each zone.

In [ ]:
from astropy import units
from dpd.driving import Route

route = Route.from_osm_relation(relation=2351006)

In [ ]:
from dpd.modeling import DistanceDataFrame, TripDataFrame

### Here is the "old way" of computing distances before we used hexogons
# points = zones.polygons_to_points()
# distance_dataframe = DistanceDataFrame.from_origins_destinations(
#     points.geometry, stops.geometry, method="distance"
# )

stops = route.stops.to_crs("EPSG:4087")
distance_dataframe = DistanceDataFrame.from_origins_destinations(
    zones_interpolated.geometry.centroid,
    stops.geometry,
    method="distance",
)
distance_dataframe.columns = stops.name
distance_dataframe

And here we can plot the number of people within 5, 10, and 15 minutes of each metro stop.

In [ ]:
from pandas import DataFrame

times = [5, 10, 15]
data = []
for time in times:
    test = lambda x: x / 1.35 < time * 60
    tdf = TripDataFrame(
        (test(distance_dataframe) * 1).to_numpy()
        * zones_interpolated["Total Population + Worker Population"].to_numpy()[
            :, None
        ],
        index=zones_interpolated.index,
        columns=stops.name,
    ).astype(int)
    data.append(tdf.sum())

DataFrame(data=data, index=times, columns=distance_dataframe.columns).T.plot(kind="bar")

And here we can plot where people are within 15 minutes of a metro stop. This could be done much easier if we simply applied .buffer to each stop geometry.

And here we will plot the number of potential riders within 15 minutes of each stop.

In [ ]:
from matplotlib import pyplot as plt

fig = plt.figure(figsize=(18, 16))
ax = fig.add_subplot(111)
(distance_dataframe / 1.35).hist(
    weights=zones_interpolated["Total Population + Worker Population"],
    range=(0, 900),
    bins=30,
    cumulative=True,
    sharey=True,
    ax=ax,
)
ax.set_ylabel("Population (cumulative)")
ax.set_xlabel("Time (seconds)")

In [ ]:
from mapclassify import Quantiles

zones_interpolated["Accessibility w/in 15 minutes"] = tdf.T.sum()
layer = SolidPolygonLayer.from_geopandas(zones_interpolated, opacity=0.1)
jc = Quantiles(zones_interpolated["Accessibility w/in 15 minutes"], k=10)
colors = []
for color in jc.yb:
    colors.append(Viridis_20.colors[color])
layer.get_fill_color = numpy.uint8(colors)
layers = [layer]

Map(layers=layers)

In [ ]:
from shapely import voronoi_polygons, MultiPoint

stops = {}
for route in network.routes:
    for index, stop in network.routes[route].stops.iterrows():
        stops[stop["name"]] = stop["geometry"]

v = voronoi_polygons(MultiPoint(list(stops.values())), extend_to=los_angeles_county)

folium_map = folium.Map(location=(34, -118.3), zoom_start=11)
vs = GeoSeries(v.geoms, crs="EPSG:4326")
vs.intersection(los_angeles_county).explore(m=folium_map)
for route in network.routes:
    network.routes[route].stops.explore(m=folium_map)
folium_map